In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm  # 1. tqdm 임포트

# RLE 디코딩/인코딩 함수 (기존과 동일)
def rle_decode(mask_rle, shape):
    if pd.isna(mask_rle) or mask_rle == '': 
        return np.zeros(shape, dtype=np.uint8)
    
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
        
    return img.reshape(shape)

def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


# 앙상블 설정
IMG_HEIGHT = 2048 
IMG_WIDTH = 2048
IMG_SHAPE = (IMG_HEIGHT, IMG_WIDTH)

#,

csv_files = [
    './outputs/swin_upernet_pseudo_9753.csv',
    './outputs/effb3_unetpp_pseudo_9749.csv',
    './outputs/effb3_deeplabv3p_pseudo_9744.csv',
    './outputs/UperNet_swinL_9732.csv',
    './outputs/experiment1618_9731.csv',
    './outputs/swin_upernet_9730.csv',
]

VOTE_THRESHOLD = 3

# 데이터 로드
print("Loading CSV files...")
df_list = [pd.read_csv(f) for f in csv_files]
merged_df = pd.concat(df_list)

# (Image, Class) 그룹별 앙상블 함수 정의
def ensemble_group(rle_series):
    vote_mask = np.zeros(IMG_SHAPE, dtype=np.float32)
    
    for rle in rle_series:
        decoded = rle_decode(rle, IMG_SHAPE)
        vote_mask += decoded
        
    # Threshold 적용 (Hard Voting)
    final_mask = (vote_mask >= VOTE_THRESHOLD).astype(np.uint8)
    
    return rle_encode(final_mask)

# 실행 (tqdm 적용)
print("Ensembling...")

# pandas에 tqdm 등록
tqdm.pandas()

# .apply() 대신 .progress_apply() 사용
result_series = merged_df.groupby(['image_name', 'class'])['rle'].progress_apply(ensemble_group)

# 결과 정리
result_df = result_series.reset_index()
result_df.columns = ['image_name', 'class', 'rle'] 

# 저장
result_df.to_csv('ensemble_submission.csv', index=False)
print("Saved to ensemble_submission.csv")
print(result_df.head())

Loading CSV files...
Ensembling...


100%|████████████████████████████████████████████████████████████| 8352/8352 [05:59<00:00, 23.26it/s]


Saved to ensemble_submission.csv
               image_name     class  \
0  image1661319116107.png  Capitate   
1  image1661319116107.png    Hamate   
2  image1661319116107.png    Lunate   
3  image1661319116107.png  Pisiform   
4  image1661319116107.png    Radius   

                                                 rle  
0  2948051 1 2950093 11 2952138 16 2954185 18 295...  
1  2964524 12 2966564 22 2968607 30 2970652 35 29...  
2  3269572 9 3271615 17 3271637 4 3273660 32 3275...  
3  3195975 3 3198022 14 3200068 18 3202109 27 320...  
4  3279664 7 3281710 11 3283757 14 3285804 16 328...  
